In [ ]:
from pyspark.sql import SparkSession

In [ ]:
print("======================= SparkSession Starting ========================")
spark = SparkSession.builder.appName("job-example").getOrCreate()

print("=========================== Set Spark Conf ===========================")
#spark.conf.set('key', 'value')

In [ ]:
print("============================== Read data =============================")
df = spark.read.format("com.crealytics.spark.excel").option("header", "true").load("airtravel.xlsx")
df.printSchema()
df.show()

In [ ]:
print("============================= Write data =============================")
df.write.mode("overwrite").format("delta").save("delta/airtravel")

In [ ]:
print("============================== Finished ==============================")
spark.stop()